
# Setup for Calibration  
 - RIGOL DL3021 as DC Load
 - RIGOL DP832 as DC Power Supply, Source (optional, only use when VAR source not populated)
 
### DC Load Setup
 - Default settings
 - Set Constant Curent mode (CC button)
 - Switch load function on (ON/OFF button, not the power button)
 
### DC Power Supply Setup
 - Script uses Channel 1
 - Set Current to 500mA
 - Set Over Voltage Protection (OVP) to off
 - Set Current limit to 500mA
 - Set Over Current Protection (OCP) to on

### Note 
 Max adc value read is around 61684

# Calibration Script

In [ ]:
# J18, J19, and J24 must be closed
# Nothing should be connected to the Target connector or J25
import pyvisa
import time
from pynq import Overlay
from foboslib.power import PowerManager
from foboslib.config.pynq_conf import IP, PORT, OVERLAY_FILE, FOBOS_HOME
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

## Error poly helper func
def gen_error_poly(theory_volts,adc_volts,verbose=False):
    # numpyfy
    setvolts = np.array(theory_volts)
    adcvolts = np.array(adc_volts)

    absolute_error = setvolts - adcvolts

    errorpoly = np.polyfit(adcvolts, absolute_error, 3)
    errorfunc = np.poly1d(errorpoly)

    corrected = adcvolts + errorfunc(adcvolts)
    corr_error = setvolts - corrected

    if (verbose):
        plt.rcParams["figure.figsize"] = (10,16)
        plt.figure()
        plt.subplots_adjust(wspace=0.5, hspace=0.5)
        plt.subplot(411)
        plt.plot(adcvolts,setvolts)
        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Setting [Volts]')
        plt.title('Measurement vs. Setting')
        plt.subplot(412)

        plt.plot(adcvolts, absolute_error)
        plt.plot(adcvolts, errorfunc(adcvolts))
        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Absolute Error')
        plt.title('Measurement Error')
        plt.subplot(413)

        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Corrected [Volts]')
        plt.title('After Correction')
        plt.plot(adcvolts, corrected)
        plt.subplot(414)

        plt.xlabel('Measurement [Volts]')
        plt.ylabel('Absolute Error')
        plt.title('Error After Correction')
        plt.plot(adcvolts, corr_error)
    return errorpoly

##### NETWORK SETTINGS #####
dcload_ip = "192.168.10.21"
powersupply_ip = "192.168.10.32"

##### CALIBRATION SETTINGS #####
MAX_MILLIAMPS = 250
STEP_SIZE = 25  

MAX_VOLTS = 3.5  # max 3.65
MIN_VOLTS = 1    # min 0.9 
STEP_VOLTS = .25 # 0.05 or multiple of this

SAMPLES_PER_STEP = 50
WAIT_SECONDS = 5
CURR_CAL_VOLT = 1
OCP_LIMIT = .5


# SCPI commands for DC Load and Power Supply
CONNECT_INSTR_STR = 'TCPIP0::{}::INSTR'
SET_SRC1_VOLT_STR = ':SOUR1:VOLT:IMM {}'
SET_SRC1_OCP_STR = ':SOUR1:CURR:PROT {}'
SET_LOAD_CURR_STR = ':SOURce:CURRent:LEVel:IMMediate {}'
GET_LOAD_CURR_STR = ':MEASure:CURRent:DC?'
GET_LOAD_VOLT_STR = ':MEASure:VOLTage:DC?'
            
            
def getInputuseSource(src, mode):
    # Helper function to get user input for enabling calibration of different sources
    while(1):
        rec = input("Connect {} OUT to DC load then enter y, or enter n to skip {} calibration:".format(src, mode))

        if (rec.strip("\n").lower() == "y"):
            return True
        if (rec.strip("\n").lower() == "n"):
            return False

def getInputUseVar():
    # Helper function to get user input for enabling external power supply
    while(1):
        rec = input("Are you using an external power source for VAR power? (y/n):")

        if (rec.strip("\n").lower() == "y"):
            return True
        if (rec.strip("\n").lower() == "n"):
            return False
        
##### Open PYNQ PowerManager #####
overlay = Overlay(OVERLAY_FILE)
pm = PowerManager(overlay, False)

##### Connect and init connection to DC Load and Power Supply #####
rm = pyvisa.ResourceManager()

# Connect
try:
    # Init
    dcload = rm.open_resource(CONNECT_INSTR_STR.format(dcload_ip))
    dcload.read_termination = '\n'
    dcload.write_termination = '\n'
    print(dcload.query('*IDN?'))

    # Set to Constant Current (CC) mode
    dcload.write(':SOUR:FUNC CURRent')
except:
    dcload.close()

# Connect
use_external_var = getInputUseVar()
if use_external_var:
    try:
        # Init
        powersupply = rm.open_resource(CONNECT_INSTR_STR.format(powersupply_ip))
        powersupply.read_termination = '\n'
        powersupply.write_termination = '\n'
        print(powersupply.query('*IDN?'))

        # Set to OCP
        powersupply.write(SET_SRC1_OCP_STR.format(OCP_LIMIT))
    except:
        dcload.close()
        print("Connecting to exteranl poewr failed!")
    
# GAINS = [200]
GAINS = [25, 50, 100, 200]
NAMES = ["VAR", "5V", "3V3"]

data = pd.DataFrame(columns=["source","gain","type","poly"])

#### Run VAR voltage calibration #####
if (getInputuseSource("VAR", "voltage")):
    # Check if using shield VAR or external power supply
    
    # Turn on power supply
    print("Running volage calibration for VAR power supply")
    if (not use_external_var):
        pm.OutVarOn()
        pm.OutVarSet(MIN_VOLTS)
    else:
        powersupply.write(SET_SRC1_VOLT_STR.format(MIN_VOLTS))
    
    # Make sure DC load is disabled
    dcload.write(SET_LOAD_CURR_STR.format(0))
    
    
    actual_volts = []
    theory_volts = [x/100 for x in list(range(int(MIN_VOLTS*100), int(MAX_VOLTS*100), int(STEP_VOLTS*100)))]
    for volt in theory_volts:
        # Set voltage output and wait for it to stabilize
        if (not use_external_var):
            pm.OutVarSet(volt)
        else:
            powersupply.write(SET_SRC1_VOLT_STR.format(volt))
        time.sleep(WAIT_SECONDS)
        
        # Get average voltage and store
        meas = 0
        for i in range(SAMPLES_PER_STEP):
            meas += pm.MeasVoltVar()
        meas = meas/SAMPLES_PER_STEP
        actual_volts.append(meas)

        print("Target Voltage", volt, "Measured Voltage:", actual_volts[-1])
    
    # Calculate error polynomial and add to datafrom
    err_poly = gen_error_poly(theory_volts, actual_volts, False)

    for GAIN in GAINS:
        ## HOTFIX: lookup uses gain so duplicate until exception is made for voltage
        line = ["VAR", GAIN, "VOLT", list(err_poly)]
        data.loc[len(data)] = line


#### Run current calibration #####
if (not use_external_var):
    pm.OutVarOn()
    pm.OutVarSet(CURR_CAL_VOLT)

for NAME in NAMES:
    # Check if user wants to calibrate current source
    if NAME == "VAR":
        cont = getInputuseSource("VAR", "current")
    elif NAME == "5V":
        cont = getInputuseSource("5V", "current")
    elif NAME == "3V3":
        cont = getInputuseSource("3V3", "current")
        
    if (cont):
        
        for GAIN in GAINS:
            
            if NAME == "VAR":
                pm.GainVarSet(GAIN)
                if (use_external_var):
                    powersupply.write(SET_SRC1_VOLT_STR.format(CURR_CAL_VOLT))
            elif NAME == "5V":
                pm.Gain5vSet(GAIN)
            elif NAME == "3V3":
                pm.Gain3v3Set(GAIN)

            print("Running current calibration for {} power supply for gain={}".format(NAME, GAIN))

            # ##### Gather samples #####
            print("Code, ADC Current, Actual Current")
            theory_amps = [x/1000 for x in list(range(0,MAX_MILLIAMPS,STEP_SIZE))]
            print("Currents steps:", theory_amps)
            
            shield_amps = []
            shield_volts = []
            load_volts   = []
            load_currs   = []
            for amp in theory_amps:
                
                # Set current load and wait for it to stabilize
                actual_curr = amp
                dcload.write(SET_LOAD_CURR_STR.format(actual_curr))
                time.sleep(WAIT_SECONDS)

                # Read voltage and current from DC load and store 
                actual_curr = float(dcload.query(GET_LOAD_CURR_STR))
                actual_volt = float(dcload.query(GET_LOAD_VOLT_STR))
                load_currs.append(amp)
                load_volts.append(actual_volt)
                
                # Read voltage and current from FOBOS shield and store
                if NAME == "VAR":
                    # Get average current
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurrVar()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)

                elif NAME == "5V":                    
                    # Get average voltage
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasVolt5v()
                    meas = meas/SAMPLES_PER_STEP
                    shield_volts.append(meas)
                    
                    # Get average current
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurr5v()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)
                elif NAME == "3V3":
                    # Get average voltage
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasVolt3v3()
                    meas = meas/SAMPLES_PER_STEP
                    shield_volts.append(meas)    
                
                    # Get average current
                    meas = 0
                    for i in range(SAMPLES_PER_STEP):
                        meas += pm.MeasCurr3v3()
                    meas = meas/SAMPLES_PER_STEP
                    shield_amps.append(meas)
                    
                print(i, ",", meas, "," ,  actual_curr)   

            # Turn off current load
            dcload.write(SET_LOAD_CURR_STR.format(0))
            
            # Calculate current error poly and add to dataframe
            cerr_poly = gen_error_poly(load_currs, shield_amps, False)
            line = [NAME, GAIN, "CURR", list(cerr_poly)]
            data.loc[len(data)] = line
            
            # Calculate voltage error poly and add to dataframe if not VAR source
            if NAME != "VAR":
                verr_poly = gen_error_poly(load_volts, shield_volts, False)
                line = [NAME, GAIN, "VOLT", list(verr_poly)]
                data.loc[len(data)] = line

    
##### Close Connections #####
if use_external_var:
    powersupply.write(SET_SRC1_VOLT_STR.format(0))
    powersupply.close()
else:
    pm.OutVarOff()
dcload.close()


##### Store Results #####
data.to_csv("{}/software/foboslib/config/power_conf.csv".format(FOBOS_HOME), index=False)   
print("done") 

# View Stored Calibration Coefficient

In [ ]:
def GetCalibration(src, mtype, gain):
    calibration_df = pd.read_csv("{}/software/foboslib/config/power_conf.csv".format(FOBOS_HOME)) 
    cond = (calibration_df['source'] == src) & (calibration_df['gain'] == gain) & (calibration_df['type'] == mtype)
    if len(calibration_df.loc[cond].values.tolist()) < 1:
        return [0,0,0,0]
    coeffs = calibration_df.loc[cond].values.tolist()[0][3]
    return coeffs

print(GetCalibration("VAR", "VOLT", 25))

print(GetCalibration("VAR", "CURR", 25))
print(GetCalibration("VAR", "CURR", 50))
print(GetCalibration("VAR", "CURR", 100))
print(GetCalibration("VAR", "CURR", 200))

print(GetCalibration("5V", "CURR", 25))
print(GetCalibration("5V", "CURR", 50))
print(GetCalibration("5V", "CURR", 100))
print(GetCalibration("5V", "CURR", 200))

print(GetCalibration("5V", "VOLT", 25))

print(GetCalibration("3V3", "CURR", 25))
print(GetCalibration("3V3", "CURR", 50))
print(GetCalibration("3V3", "CURR", 100))
print(GetCalibration("3V3", "CURR", 200))

print(GetCalibration("3V3", "VOLT", 25))

# Verify Calibration

In [ ]:
import pyvisa
import time
from pynq import Overlay
from foboslib.config.pynq_conf import IP, PORT, OVERLAY_FILE
from foboslib.power import PowerManager


ENABLE_CALIBRATION = True
SRC = "VAR"
GAIN = 50

## Connect and init connection to DC Load
rm = pyvisa.ResourceManager()
rm.list_resources()
my_instrument = rm.open_resource('TCPIP0::192.168.10.21::INSTR')
my_instrument.read_termination = '\n'
my_instrument.write_termination = '\n'
print(my_instrument.query('*IDN?'))

# Set to Constant Current (CC) mode
my_instrument.write(':SOUR:FUNC CURRent')
print(my_instrument.query(':SOUR:FUNC?'))


overlay = Overlay(OVERLAY_FILE)
pm = PowerManager(overlay, ENABLE_CALIBRATION)

pm.Reset()
pm.OutVarOn()
pm.OutVarSet(1)

pm.GainVarSet(GAIN)


SAMPLES_PER_STEP = 25
SLEEP_TIME = 1
try:
    #### START DC LOAD SCRIPT
    currents = [.05,.15,.2,.25,.2,.15,.05]
    pm.TrigSwEnOn()
    for curr in currents:
        my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(curr))
        time.sleep(SLEEP_TIME)
        print("=======")
        if SRC == "VAR":
            curr = pm.MeasCurrVar()
            volt = pm.MeasVoltVar()
        elif SRC == "3V3":
            curr = pm.MeasCurr3v3()
            volt = pm.MeasVolt3v3()
        elif SRC == "5V":
            curr = pm.MeasCurr5v()
            volt = pm.MeasVolt5v()



        print("Load Current:", my_instrument.query(':MEASure:CURRent:DC?'), "Measured Current:", curr)
        print("Load Voltage:", my_instrument.query(':MEASure:VOLTage:DC?'), "Measured Voltage:", volt)
    pm.TrigSwEnOff()
    #### END DC LOAD SCRIPT
    my_instrument.write(':SOURce:CURRent:LEVel:IMMediate {}'.format(0))
    pm.OutVarOff()

    if SRC == "VAR":
        avg_curr = pm.MeasAvgCurrVar()
        avg_volt = pm.MeasAvgVoltVar()
        max_curr = pm.MeasMaxCurrVar()
        max_volt = pm.MeasMaxVoltVar()
    elif SRC == "3V3":
        avg_curr = pm.MeasAvgCurr3v3()
        avg_volt = pm.MeasAvgVolt3v3()
        max_curr = pm.MeasMaxCurr3v3()
        max_volt = pm.MeasMaxVolt3v3()
    elif SRC == "5V":
        avg_curr = pm.MeasAvgCurr5v()
        avg_volt = pm.MeasAvgVolt5v()
        max_curr = pm.MeasMaxCurr5v()
        max_volt = pm.MeasMaxVolt5v()


    print("Avg curr:", avg_curr, "Avg volt:", avg_volt)
    print("Max curr:", max_curr, "Max volt:",max_volt)
    print("expected:", sum(currents)/len(currents))  
except:
    my_instrument.close()
    

In [ ]:
Avg curr: 0.12402532997634852 Avg volt: 0.8585488670176241
Max curr: 0.22272068360418099 Max volt: 0.884794384679942

TODO:
- [X] No difference for gains in voltage calibration
- [X] Use $FOBOS_HOME/software for path to calibration in power.py
- [X] Move calibration CSV to same folder as pynq_conf 
- [x] Create requirement-pynq.txt for [pandas, pyvisa] AND call from install script
- [x] Check that lack of CSV does not cause power.py to error out
- [x] 5v/3v3 calibration over shunt once wire is ready (sense cables to get better V measurement)
- [ ] Move script to command line
- [ ] In MeasAvg<> check max, if it is ADC max then the measurement was clipped so return None. Add error log to explain why None was return
- [x] Voltage calibration is off
- [ ] confirm gain correction of 3v3 and 5v is correct



In [ ]:
import pyvisa
import time

load_ip = "192.168.10.21"
powersupply_ip = "192.168.10.32"

CONNECT_INSTR_STR = 'TCPIP0::{}::INSTR'
SET_SRC1_VOLT_STR = ':SOUR1:VOLT:IMM {}'
SET_SRC1_OCP_STR = ':SOUR1:CURR:PROT {}'


## Connect and init connection to DC Load


rm = pyvisa.ResourceManager()
rm.list_resources()
powersupply = rm.open_resource('TCPIP0::{}::INSTR'.format(powersupply_ip))
powersupply.read_termination = '\n'
powersupply.write_termination = '\n'
print(powersupply.query('*IDN?'))

## set OCP
powersupply.write(SET_SRC1_OCP_STR.format(.55))

## Set Vout on CH1
powersupply.write(SET_SRC1_VOLT_STR.format(2.5))

powersupply.close()
    